# Import Data

In [1]:
from RL_bidding import *
from utility import *
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np, pickle
import math
from xgboost import XGBClassifier
from sklearn.externals import joblib
import pandas as pd

In [2]:
model = joblib.load("xgb.pickle.dat")
valid = load_pickle('valid_df_after_preprocessing.pkl')
# valid_X = valid.drop(['click', 'bidprice', 'payprice'], axis = 1)
# valid_pCTR = model.predict_proba(valid_X)[:,1]

# test = load_pickle('test_df_after_preprocessing.pkl')
# test_pCTR = model.predict_proba(test)[:,1]
# train_raw = pd.read_csv('train.csv')
# m_pdf = calc_m_pdf(train_raw['payprice'].tolist())

# with open('m_pdf.pickle', 'wb') as handle:
#     pickle.dump(m_pdf, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('valid_pCTR.pickle', 'wb') as handle:
#     pickle.dump(valid_pCTR, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_pCTR.pickle', 'wb') as handle:
#     pickle.dump(test_pCTR, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('m_pdf.pickle', 'rb') as handle:
    m_pdf = pickle.load(handle)
    
with open('valid_pCTR.pickle', 'rb') as handle:
    valid_pCTR = pickle.load(handle)

with open('test_pCTR.pickle', 'rb') as handle:
    test_pCTR = pickle.load(handle)


# reinforcement learning on validation set 

In [3]:
# batch_start_index = N_per_batch*current_batch
# batch_end_index = N_per_batch*(current_batch+1)
# auction_in = valid[ batch_start_index : batch_end_index]
# pCTR = valid_pCTR[ batch_start_index : batch_end_index ]

# rlb_dp_i = RLB_DP_I(auction_in, train_cpm, train_thetaAvg )

# rlb_dp_i.calc_optimal_value_function_with_approximation_i(N_per_batch, B_per_batch, max_market_price, m_pdf,\
#                                                           './log/V_out_N={}_batch={}.txt'.format(N_per_batch, current_batch))

# rlb_dp_i.load_value_function(N_per_batch, B_per_batch, './log/V_out_N={}_batch={}.txt'.format(N_per_batch, current_batch))

# (bid_price, auction, imp, clk, cost) = rlb_dp_i.run(B_per_batch, auction_in, pCTR, bid_log_path, N_per_batch, max_market_price,\
#                                                     save_log=True, evaluate = True)



In [26]:
N_per_batch = 100
# train_thetaAvg = 0.0007375623256619447
train_thetaAvg = 0.5
total_batch = math.ceil( len(valid)/N_per_batch )
max_market_price = 300
B_per_batch = math.ceil( 6250000/total_batch )

target_bid_price = []
total_clks = 0
print('total_episode:{}'.format(total_batch))

total_episode:3040


In [27]:
t = Time_Tracking()

In [29]:
t.start_tracking()

for current_batch in range(0, total_batch+1, 1):
    log_in = open("./log/N={}_B={}_Batch={}.txt".format(N_per_batch, B_per_batch , current_batch), "w")
    print("logs in {}".format(log_in.name))
    
    log = "{}\t {}\t {}\t {}\t {}\t {}".format("setting", "auction","impression", "click", "cost", "win-rate", "CPM", "eCPC")
    print(log)
    log_in.write(log + "\n") 
    
    batch_start_index = N_per_batch*current_batch
    batch_end_index = N_per_batch*(current_batch+1)    
    auction_in = valid[ batch_start_index : batch_end_index]
    pCTR = valid_pCTR[ batch_start_index : batch_end_index ]
    
    rlb_dp_i = RLB_DP_I(auction_in, train_thetaAvg )

    rlb_dp_i.calc_optimal_value_function_with_approximation_i(N_per_batch, B_per_batch, max_market_price, m_pdf,\
                                                              './log/V_out_N={}_batch={}.txt'.format(N_per_batch, current_batch))

    rlb_dp_i.load_value_function(N_per_batch, B_per_batch, './log/V_out_N={}_batch={}.txt'.format(N_per_batch, current_batch))


    setting = "N={}_batch={}" .format(N_per_batch, current_batch)
    bid_log_path = "./log/bid_log/{}.txt".format(setting)

    (bid_price, auction, imp, clk, cost) = rlb_dp_i.run(B_per_batch, auction_in, pCTR, bid_log_path, N_per_batch, 300,\
                                                        save_log=True, evaluate = True)
    target_bid_price.extend(bid_price)
    total_clks += clk
    
    win_rate = imp / auction * 100
    cpm = (cost / 1000) / imp * 1000
    ecpc = (cost / 1000) / clk
    log = "{}\t {}\t {}\t {}\t {}\t {}\t {}\t {}\t".format(setting, auction, imp, clk, cost, win_rate, cpm, ecpc)
    print(log)
    log_in.write(log + "\n")
    print('current total clicks:', total_clks)
    t.stop_tracking()
    t.start_tracking()

logs in ./log/N=100_B=2056_Batch=0.txt
setting	 auction	 impression	 click	 cost	 win-rate


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in double_scalars


N=100_batch=0	 100	 46	 0	 2011	 46.0	 43.71739130434783	 inf	
current total clicks: 0
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=1.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=1	 100	 45	 0	 1976	 45.0	 43.91111111111111	 inf	
current total clicks: 0
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=2.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2	 100	 45	 0	 1935	 45.0	 43.0	 inf	
current total clicks: 0
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=3.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=3	 100	 47	 0	 2017	 47.0	 42.91489361702128	 inf	
current total clicks: 0
Time used: 0.13  minutes
logs in ./log/N=100_B=2056_Batch=4.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=4	 100	 46	 0	 1999	 46.0	 43.45652173913044	 inf	
current total clicks: 0
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=5.txt
setting	 auction	 impression	 click	 cost	 win-rat

N=100_batch=41	 100	 46	 0	 1899	 46.0	 41.28260869565217	 inf	
current total clicks: 6
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=42.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=42	 100	 46	 0	 1921	 46.0	 41.76086956521739	 inf	
current total clicks: 6
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=43.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=43	 100	 44	 0	 1861	 44.0	 42.29545454545455	 inf	
current total clicks: 6
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=44.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=44	 100	 45	 0	 2014	 45.0	 44.755555555555546	 inf	
current total clicks: 6
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=45.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=45	 100	 43	 0	 1930	 43.0	 44.883720930232556	 inf	
current total clicks: 6
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=46.txt
setting	 auction	 impress

N=100_batch=81	 100	 44	 0	 1985	 44.0	 45.11363636363636	 inf	
current total clicks: 7
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=82.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=82	 100	 40	 0	 1925	 40.0	 48.125	 inf	
current total clicks: 7
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=83.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=83	 100	 47	 0	 1903	 47.0	 40.48936170212766	 inf	
current total clicks: 7
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=84.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=84	 100	 44	 1	 1815	 44.0	 41.25	 1.815	
current total clicks: 8
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=85.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=85	 100	 47	 0	 1790	 47.0	 38.08510638297872	 inf	
current total clicks: 8
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=86.txt
setting	 auction	 impression	 click	 cost	 win-r

...

N=100_batch=2904	 100	 43	 0	 1910	 43.0	 44.41860465116279	 inf	
current total clicks: 123
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2905.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2905	 100	 49	 0	 1768	 49.0	 36.08163265306123	 inf	
current total clicks: 123
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2906.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2906	 100	 50	 0	 1821	 50.0	 36.42	 inf	
current total clicks: 123
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2907.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2907	 100	 49	 0	 1763	 49.0	 35.97959183673469	 inf	
current total clicks: 123
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2908.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2908	 100	 50	 1	 2004	 50.0	 40.08	 2.004	
current total clicks: 124
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2909.txt
setting	 auction	 i

N=100_batch=2944	 100	 45	 0	 2009	 45.0	 44.644444444444446	 inf	
current total clicks: 127
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=2945.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2945	 100	 49	 0	 1958	 49.0	 39.95918367346939	 inf	
current total clicks: 127
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=2946.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2946	 100	 49	 0	 1878	 49.0	 38.326530612244895	 inf	
current total clicks: 127
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=2947.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2947	 100	 48	 0	 2003	 48.0	 41.72916666666667	 inf	
current total clicks: 127
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=2948.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2948	 100	 47	 0	 1940	 47.0	 41.276595744680854	 inf	
current total clicks: 127
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=294

N=100_batch=2983	 100	 44	 0	 1892	 44.0	 43.0	 inf	
current total clicks: 130
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2984.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2984	 100	 49	 0	 1988	 49.0	 40.57142857142857	 inf	
current total clicks: 130
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2985.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2985	 100	 52	 0	 1966	 52.0	 37.80769230769231	 inf	
current total clicks: 130
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2986.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2986	 100	 49	 0	 1967	 49.0	 40.142857142857146	 inf	
current total clicks: 130
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2987.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=2987	 100	 48	 0	 1794	 48.0	 37.375	 inf	
current total clicks: 130
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=2988.txt
setting	 auction	 im

N=100_batch=3023	 100	 49	 0	 1958	 49.0	 39.95918367346939	 inf	
current total clicks: 132
Time used: 0.11  minutes
logs in ./log/N=100_B=2056_Batch=3024.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=3024	 100	 52	 0	 1812	 52.0	 34.84615384615385	 inf	
current total clicks: 132
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=3025.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=3025	 100	 40	 0	 1898	 40.0	 47.449999999999996	 inf	
current total clicks: 132
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=3026.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=3026	 100	 44	 0	 1981	 44.0	 45.02272727272727	 inf	
current total clicks: 132
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=3027.txt
setting	 auction	 impression	 click	 cost	 win-rate
N=100_batch=3027	 100	 43	 0	 2016	 43.0	 46.88372093023256	 inf	
current total clicks: 132
Time used: 0.12  minutes
logs in ./log/N=100_B=2056_Batch=3028.

ZeroDivisionError: division by zero

In [ ]:
# log_in.flush()
# log_in.close()